# workflow 开发测试

## 环境安装

In [ ]:
# 更新 pip
!/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python -m pip install --upgrade pip

In [ ]:
# 下载安装包
!rm modelarts*.whl
!wget -N https://modelarts-cnnorth4-market.obs.cn-north-4.myhuaweicloud.com/workflow-apps/modelarts-1.3.0-py2.py3-none-any.whl
!wget -N https://modelarts-cnnorth4-market.obs.cn-north-4.myhuaweicloud.com/workflow-apps/v0.1.1/85719177/modelarts_workflow-0.1.1-py2.py3-none-any.whl

In [1]:
# 安装
#!pip uninstall -y modelarts modelarts-workflow
!pip install modelarts-*-py2.py3-none-any.whl
!pip install modelarts_workflow-*-py2.py3-none-any.whl
!pip install modelarts-*-py2.py3-none-any.whl
!pip install modelarts_workflow-*-py2.py3-none-any.whl

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Processing ./modelarts-1.3.0-py2.py3-none-any.whl
     |████████████████████████████████| 76 kB 92.0 MB/s  eta 0:00:01
     |████████████████████████████████| 138 kB 19.1 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 46.1 MB/s  eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
  Attempting uninstall: configparser
    Found existing installation: configparser 5.2.0
    Uninstalling configparser-5.2.0:
      Successfully uninstalled configparser-5.2.0
  Attempting uninstall: m

## 导入模块

注意: 安装好 pip 包后需要重启内核.

In [2]:
import os
from modelarts import workflow as wf

INFO:matplotlib.font_manager:generated new fontManager


# 创建 workflow

In [3]:
# 定义统一存储对象管理输出目录
output_storage = wf.data.OutputStorage(name="output_storage", description="输出目录统一配置")

# 创建标注任务
data = wf.data.DatasetPlaceholder(name="input_data")

label_step = wf.steps.LabelingStep(
    name="labeling", title="标注图像",
    properties=wf.steps.LabelTaskProperties(
        task_type=wf.data.LabelTaskTypeEnum.IMAGE_SEGMENTATION,
        task_name=wf.Placeholder(name="task_name", placeholder_type=wf.PlaceholderType.STR, 
                                 description="请输入一个只包含大小写字母、数字、下划线、中划线或者中文字符的名称。\
                                              填写已有标注任务名称，则直接使用该标注任务；填写新标注任务名称，则自动创建新的标注任务")),
    inputs=wf.steps.LabelingInput(name="labeling_input", data = data),
    outputs=wf.steps.LabelingOutput(name="labeling_output"),
)
# 对标注任务进行发布
release_step = wf.steps.ReleaseDatasetStep(
    name="release",
    title="发布数据集",
    inputs=wf.steps.ReleaseDatasetInput(name="input_data", data=label_step.outputs["labeling_output"].as_input()),
    outputs=wf.steps.ReleaseDatasetOutput(name="labeling_output"),
    depend_steps=[label_step]
)

In [4]:
# 创建训练作业
job_step = wf.steps.JobStep(
    name="training_job",
    title="训练模型",
    algorithm = wf.AIGalleryAlgorithm( # 从 AI Gallery 中导入.
    #algorithm = wf.Algorithm( # 从算法中导入.
        #algorithm_id = "87a49f4b-eca2-4072-8b31-933a5f4c7989",# 从算法中导入.
        
        subscription_id="44297f21-299c-4040-9a11-64c30bac449b", # 订阅算法的ID
        item_version_id="Go9ZZo", # 订阅算法的版本ID
        
        parameters = [
            wf.AlgorithmParameters(name="learning_rate_strategy", value=wf.Placeholder(name="learning_rate_strategy", placeholder_type=wf.PlaceholderType.STR, default="0.002", description="训练的学习率策略(10:0.001,20:0.0001代表0-10个epoch学习率0.001，10-20epoch学习率0.0001),如果不指定epoch, 会根据验证精度情况自动调整学习率，并当精度没有明显提升时，训练停止")),
            wf.AlgorithmParameters(name="batch_size", value=wf.Placeholder(name="batch_size", placeholder_type=wf.PlaceholderType.INT, default=64, description="每步训练的图片数量（单卡）")),
        ]
    ),
    inputs = [wf.steps.JobInput(name = "data_url", data = release_step.outputs["labeling_output"].as_input()), 
              #wf.steps.JobInput(name = "train_url", data = wf.data.obs.OBSPath(obs_path = output_storage.join("/train_output/"))),
             ],
    
    outputs = [wf.steps.JobOutput(name = "train_url", obs_config = wf.data.OBSOutputConfig(obs_path=output_storage.join("/train_output/")))],
    
    spec = wf.steps.JobSpec(resource = wf.steps.JobResource(flavor = wf.Placeholder(name = "training_flavor", placeholder_type = wf.PlaceholderType.JSON, description = "训练资源规格")),
                            log_export_path=wf.steps.job_step.LogExportPath(obs_url=output_storage.join("/logs/"))
                           ),
    depend_steps = [release_step]
)

model_name = wf.Placeholder(name="model_name", placeholder_type=wf.PlaceholderType.STR, description="请输入一个1至64位且只包含大小写字母、中文、数字、中划线或者下划线的名称。工作流第一次运行建议填写新的模型名称，后续运行会自动在该模型上新增版本")


In [5]:
# 模型注册
model_step = wf.steps.ModelStep(
    name="model_step",
    title="注册模型",
    inputs=[wf.steps.ModelInput(name="model_input", data=job_step.outputs["train_url"].as_input())],
    outputs=[wf.steps.ModelOutput(name="model_output", model_config=wf.steps.ModelConfig(model_name=model_name, model_type="PyTorch"))],
    depend_steps=[job_step]
)
# 服务部署
service_step = wf.steps.ServiceStep(
    name="service_step",
    title="部署服务",
    inputs=[wf.steps.ServiceInput(name="service_input", data=wf.data.ServiceInputPlaceholder(name="service_model", model_name=model_name))],
    outputs=[wf.steps.ServiceOutput(name="service_output")],
    depend_steps=[model_step]
)
# 构建工作流对象
workflow = wf.Workflow(name="DeepLabV3plus", # DeepLabV3plus
                       desc="workflow 图像分割算法",
                       steps=[label_step, release_step, job_step, model_step, service_step], storages=[output_storage])

In [6]:
# 发布后, 若日志打印显示发布成功，则可前往MA的workflow页面中查看新发布的工作流。
workflow.release()

INFO:root:start releasing Workflow DeepLabV3plus
INFO:root:Workflow DeepLabV3plus successfully released, and workflow ID is 4a1e3f42-bc64-47c7-9d3b-eb4624a92af1


'4a1e3f42-bc64-47c7-9d3b-eb4624a92af1'

# 发布到 AI Gallery

In [ ]:
# 新建Workflow资产
content_id = workflow.create_content(title="DeepLab v3 plus 图像语义分割算法", visibility="public")
print(content_id)

In [7]:
# 发布Workflow的新版本
idd = "e4cdd5bd-fef9-4325-a9df-b8e702501a96"
workflow.release_to_gallery(content_id = idd, version="1.0.0", desc="DeepLab v3 plus 图像语义分割算法")

INFO:root:Start publish Workflow DeepLabV3plus to the AI gallery, and version name is 1.0.0
INFO:root:Workflow version 1.0.0 successfully published to the AI gallery
